<a href="https://colab.research.google.com/github/samik1986/HF_learn/blob/main/Test_HF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 69.8 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.56.1
    Uninstalling transformers-4.56.1:
      Successfully uninstalled transformers-4.56.1


In [2]:
from transformers import AutoModel

In [3]:
bert_model = AutoModel.from_pretrained('bert-base-uncased')
print(type(bert_model))

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

<class 'transformers.models.bert.modeling_bert.BertModel'>


In [6]:
from transformers import BertConfig, BertModel

bert_config = BertConfig.from_pretrained('bert-base-uncased')
bert_model = BertModel(bert_config)
print(type(bert_config))

<class 'transformers.models.bert.configuration_bert.BertConfig'>


In [7]:
bert_config, bert_model

(BertConfig {
   "architectures": [
     "BertForMaskedLM"
   ],
   "attention_probs_dropout_prob": 0.1,
   "classifier_dropout": null,
   "gradient_checkpointing": false,
   "hidden_act": "gelu",
   "hidden_dropout_prob": 0.1,
   "hidden_size": 768,
   "initializer_range": 0.02,
   "intermediate_size": 3072,
   "layer_norm_eps": 1e-12,
   "max_position_embeddings": 512,
   "model_type": "bert",
   "num_attention_heads": 12,
   "num_hidden_layers": 12,
   "pad_token_id": 0,
   "position_embedding_type": "absolute",
   "transformers_version": "4.56.2",
   "type_vocab_size": 2,
   "use_cache": true,
   "vocab_size": 30522
 },
 BertModel(
   (embeddings): BertEmbeddings(
     (word_embeddings): Embedding(30522, 768, padding_idx=0)
     (position_embeddings): Embedding(512, 768)
     (token_type_embeddings): Embedding(2, 768)
     (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
     (dropout): Dropout(p=0.1, inplace=False)
   )
   (encoder): BertEncoder(
     (layer): Mo

In [12]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
token = tokenizer.tokenize('I have a new GPU!')
input_ids = tokenizer.convert_tokens_to_ids(token)
print(input_ids)

final_inputs = tokenizer.prepare_for_model(input_ids)
print(final_inputs)
print(tokenizer.decode(final_inputs['input_ids']))
# print(token)
# print(type(tokenizer))

[1045, 2031, 1037, 2047, 14246, 2226, 999]
{'input_ids': [101, 1045, 2031, 1037, 2047, 14246, 2226, 999, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}
[CLS] i have a new gpu! [SEP]


In [19]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenizer.pad_token_type_id
sentences = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!",
]
tokens = [tokenizer.tokenize(sentence) for sentence in sentences]
print(tokens)
ids = [tokenizer.convert_tokens_to_ids(token) for token in tokens]
print(ids[0])
print(ids[1])

[['i', "'", 've', 'been', 'waiting', 'for', 'a', 'hugging', '##face', 'course', 'my', 'whole', 'life', '.'], ['i', 'hate', 'this', 'so', 'much', '!']]
[1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012]
[1045, 5223, 2023, 2061, 2172, 999]


In [27]:
import torch
import torch.nn.functional as F

In [62]:
idst = [torch.tensor([ids[0]]), torch.tensor([ids[1]])]
idst1 = [torch.tensor(ids[0]), torch.tensor(ids[1])]
# idst[1] = torch.tensor(ids[1])

max_rows = max(tensor.size(0) for tensor in idst1)

# Pad the tensors to make their first dimension equal
padded_data = [F.pad(tensor, (0, max_rows - tensor.size(0))) for tensor in idst1]

all_ids = torch.stack(padded_data)
# print(padded_data)
print(all_ids)


tensor([[ 1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,  2607,
          2026,  2878,  2166,  1012],
        [ 1045,  5223,  2023,  2061,  2172,   999,     0,     0,     0,     0,
             0,     0,     0,     0]])


In [67]:
attention_mask = torch.tensor([
    [1] * len(ids[0]) + [0] * (max_rows - len(ids[0])),
    [1] * len(ids[1]) + [0] * (max_rows - len(ids[1])),
])

In [68]:
attention_mask

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])

In [69]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
print(model(idst[0]).logits)
print(model(idst[1]).logits)
print(model(all_ids).logits)

output = model(all_ids, attention_mask=attention_mask)
print(output.logits)

tensor([[-2.7276,  2.8789]], grad_fn=<AddmmBackward0>)
tensor([[ 3.1931, -2.6685]], grad_fn=<AddmmBackward0>)
tensor([[-2.7276,  2.8789],
        [ 2.5423, -2.1265]], grad_fn=<AddmmBackward0>)
tensor([[-2.7276,  2.8789],
        [ 3.1931, -2.6685]], grad_fn=<AddmmBackward0>)
